### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2018-10-08 16:10:13--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6022:1::a27d:4201, 162.125.66.1
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6022:1::a27d:4201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [following]
--2018-10-08 16:10:13--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Reusing existing connection to [www.dropbox.com]:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucab742d52f0577222d52225311b.dl.dropboxusercontent.com/cd/0/get/ASf_ZQDy7RPAeItwr2PM49U55VjQapTXbx4OyXyNPD-Y-hJWVTaa8dXKNw5PVn2LE8LRRzaiB2MYa8JcnNurH5KR0cArx8yC401wqhgQXNFuNCszm_hri6Z74gvdCfTbpFexc4eOWWX0B6gYuOB2l-RX1uFCThNwR6l4NA59UCdffdZvCXC1pHhLjS24lJh89-4/file?dl=1 [following]
--2018-10-08 16:10:14--  https://ucab742d52f0577222d52225311b.dl.dropboxusercontent.com/cd/0/get/ASf

,author,day,id,link,month,summary,tag,title,year
21434,"[{'name': 'O. V. Palagin'}, {'name': 'K. S. Ma...",17,1802.06768v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Creation of the information systems and tools ...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Design and software implementation of subsyste...,2018
38436,"[{'name': 'Dong Nguyen'}, {'name': 'Barbara Mc...",22,1712.08647v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,The Internet facilitates large-scale collabora...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...","Emo, Love, and God: Making Sense of Urban Dict...",2017
25597,"[{'name': 'Zhenzhong Lan'}, {'name': 'Alexande...",15,1510.04565v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,We address the problem of generating video fea...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Beyond Spatial Pyramid Matching: Space-time Ex...,2015
3988,"[{'name': 'Yang Yu'}, {'name': 'Zhi-Hua Zhou'}]",28,0905.4570v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,An updated version will be uploaded later.,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Weak Evolvability Equals Strong Evolvability,2009
35741,[{'name': 'Dov M. Gabbay'}],1,1203.0220v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,We introduce a family of new equational semant...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",The Equational Approach to CF2 Semantics,2012


In [4]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [6]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()
lines = [" ".join(tokenizer.tokenize(x.lower())) for x in lines]

In [7]:
lines[0]

'dual recurrent attention units for visual question answering ; we propose an architecture for vqa which utilizes recurrent layers to generate visual and textual attention . the memory characteristic of the proposed recurrent attention units offers a rich joint embedding of visual and textual features and enables the model to reason relations between several parts of the image and question . our single model outperforms the first place winner on the vqa 1 . 0 dataset , performs within margin to the current state - of - the - art ensemble model . we also experiment with replacing attention mechanisms in other state - of - the - art models with our implementation and show increased accuracy . in both cases , our recurrent attention mechanism improves performance in tasks requiring sequential or relational reasoning on the vqa dataset .'

In [8]:
sorted(lines, key=len)[0]

'differential contrastive divergence ; this paper has been retracted .'

In [9]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_{t + 1} \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [58]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"


def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    UNKS = [UNK] * (n - 1)
    counts = defaultdict(Counter)
    for line in lines:
        #line = line.replace(" !? ", " !? _EOS_ ")
        #line = line.replace(" ?! ", " ?! _EOS_ ")
        #line = line.replace(" ? ", " ? _EOS_ ")
        #line = line.replace(" . ", " . _EOS_ ")
        #line = line.replace(" ! ", " ! _EOS_ ")
        #if line[-1] in {"!", ".", "?"}:
        line = line.split()
        tokens = UNKS + line + [EOS]
        # print(line)
        for i in range(n - 1, len(tokens)):
            prefix = tokens[i - n + 1: i]
            counts[tuple(prefix)][tokens[i]] += 1  
    return counts

In [59]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [15]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix, counter in counts.items():
            sum_prefix = float(np.sum([counter[key] for key in counter.keys()]))
            for token in counter.keys():
                self.probs[prefix][token] = counts[prefix][token] / sum_prefix
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [16]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [17]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [18]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ temperature
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    next_token_probs = lm.get_possible_next_tokens(prefix)
    next_tokens = list(next_token_probs)
    
    if temperature == 0:
        return next_tokens[np.argmax([next_token_probs[token] for token in next_tokens])]
    
    probas = np.array([next_token_probs[token] ** (1 / temperature) for token in next_tokens])
    probas /= np.sum(probas)
    return next_tokens[np.argmax(np.random.multinomial(1, probas))]

In [19]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [20]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial intelligence . we also provide general capabilities for high - dimensional convolutional neural network , in theory and practice of logic programs under stable model semantics and subsequently non - convex optimization with grid ( like " kicking ball "? we argue that including appropriate structure greatly improves the performance of the time series using the real world graphs that form a graph , but that the peak point of time series data , and 3 ) only . specifically , there has been that the sequential execution on mobile devices . in this paper , we show how our


In [21]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the two domains . in this paper presents a new framework for the development of a given automatically labelled document . we present a new algorithm for the purpose of this algorithm . the second one is the first time , the proposed method is compared to the best of our method is able to learn a nonlinear generalization of the art in terms of the data . _EOS_


In [22]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the source data . the proposed method achieves state - of - the - art methods . _EOS_


In [23]:
prefix = 'new method' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

new method , which is a fundamental problem in artificial intelligence ( ai ) systems , which is the first time , the proposed method is based on a large margin , and the second algorithm is proposed to deal with the same time , with a large number of features to a new algorithm for the task of learning a regression model to predict the future . _EOS_


### Evaluating language models: perplexity

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [24]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    N = 0
    perplexity = 0.
    for line in lines:
        splitted_line = line.split()
        splitted_line.append('_EOS_')
        current_prefix = ""
        for word in splitted_line:
            probability = lm.get_next_token_prob(current_prefix, word)
            perplexity += max(min_logprob, np.log(probability))
            current_prefix += word + " "
            N += 1
    
    return np.exp(perplexity * (-1./N))

In [25]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [26]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28774
N = 3, Perplexity = 61999196259042902147072.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distrivution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [86]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        NGramLanguageModel.__init__(self, lines, n)
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super(LaplaceLanguageModel, self).get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super(LaplaceLanguageModel, self).get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

In [89]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [29]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 981.53856
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [30]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} \max(0, Count(prefix_{n-1}, \hat w))} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement KneserNeyLanguageModel
- test it on 1-3 gram language models
- find optimal (within one order of magnitude) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [98]:
def ComputeKneserAbsolute(lines, counts_bigrams):
    diff_bigrams = sum([len(counter.values()) for counter in counts_bigrams.values()])
    
    counts_prefixes = defaultdict(float)
    for prefix, prefix_counter in counts_bigrams.items():
        for token in prefix_counter.keys():
            counts_prefixes[token] += 1.
    
    kneiwer_absolute_probs = defaultdict(Counter)
    for token in counts_prefixes.keys():
        kneiwer_absolute_probs[tuple()][token] = counts_prefixes[token] / diff_bigrams
    
    return kneiwer_absolute_probs

In [149]:
class KneserNeyLanguageModel(NGramLanguageModel, object): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        
        self.n = n
        assert n > 0
        bigramms = count_ngrams(lines, 2)
        self.vocab = set(token for token_counts in bigramms.values() for token in token_counts)
        NGramLanguageModel.__init__(self, lines, n)
        self.probs = {i : defaultdict(Counter) for i in range(1, self.n + 1)}
        self.probs[1] = ComputeKneserAbsolute(lines, bigramms)
        self.vocab2 = set()
        
        for n_ in range(2, self.n + 1):
            if n_ == 2:
                current_counts = bigramms
            else:
                current_counts = count_ngrams(lines, n)
            for prefix in current_counts.keys():
                tokens_of_prefix = current_counts[prefix]
                num_tokens_after_prefix = sum(tokens_of_prefix.values())
                first_additives = Counter()
                second_additives = Counter()
                
                for token in tokens_of_prefix:
                    first_additives[token] = max(0, tokens_of_prefix[token] - delta) / num_tokens_after_prefix
                    second_additives[token] = self.probs[n_ - 1][prefix[1:]][token]
                    
                lambda_prefix = (1 - sum(first_additives.values())) / sum(second_additives.values())
                
                for token in tokens_of_prefix:
                    self.probs[n_][prefix][token] = first_additives[token] + lambda_prefix * second_additives[token]
                    
                # print(sum([self.probs[n_][prefix][token] for token in tokens_of_prefix]))
        
    def get_possible_next_tokens(self, prefix):
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[self.n][tuple(prefix)]
        
    def get_next_token_prob(self, prefix, next_token):
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

In [151]:
#test that it's a valid probability model
for n in [1, 2, 3]:
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [157]:
delta_to_score = dict()
for delta in [0.1, 0.5, 1, 10, 100]:
    lm = KneserNeyLanguageModel(train_lines, n=3, delta=delta)
    ppx = perplexity(lm, test_lines)
    delta_to_score[delta] = ppx
    print("D = %.1f, Perplexity = %.5f" % (delta, ppx))

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


D = 0.1, Perplexity = 61257920485804298207232.00000
D = 0.5, Perplexity = 59515047017334379118592.00000
D = 1.0, Perplexity = 61052142906074780925952.00000
D = 10.0, Perplexity = 67651661650147164553216.00000
D = 100.0, Perplexity = 79450878463933795008512.00000
